Game Theory I: Normal Form
======================

Shane Steinert-Threlkeld  
http://www.shane.st  
S.N.M.Steinert-Threlkeld AT uva DOT nl

In [1]:
%%HTML
<style type="text/css">
.rendered_html tbody tr td:first-child {
    border-right: 1px solid black;
}
    
.rendered_html table {
    font-size: 28px;
}
</style>

What is game theory?
-----

![AlphaGo Nature paper](imgs/AlphaGo_cover.jpg)

credit: Nature Publishing Group

![Robben in the 2014 World Cup quarter-finals](imgs/robben_shootout.png)

credit: EPA/Ali Haider

In [2]:
%%HTML
<iframe src="https://giphy.com/embed/uH27sheFg9One" width="600" height="360" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/stanley-kubrick-1964-dr-strangelove-uH27sheFg9One">via GIPHY</a></p>

![One of Google's self-driving cars](imgs/google_car.jpg)

Source: Google

What is game theory?
------

* The study of strategic interaction.
* Essentially: _multi-agent decision theory_.

The Shootout
------

![Robben in the 2014 World Cup quarter-finals](imgs/robben_shootout.png)

credit: EPA/Ali Haider

The Shootout in Normal Form
-------

| &#160; | dive left | dive right |
| ----- | ----- | ----- |
| shoot left | 1, -1 | -1, 1 |
| shoot right | -1, 1 | 1, -1 |

Definition of a (Normal-Form) Game
------

A finite, $n$-person _normal form game_ (aka strategic form) is a tuple $\langle N, \{ A_i \}, \{ u_i \} \rangle$:

* $N$ is the number of players.  We represent them as integers $1, \dots , n$.

* $A_i$ is the set of _actions_ available to player $i$.

    Write $A = A_1 \times \cdots \times A_n$ for the set of _action profiles_: a specification of on action for each player.
    
    For reasons that will become clear, we will also call actions _pure strategies_ (and mutatis mutandis for profiles thereof).  We will also use $S_i$ and $S$ for the sets of strategies and profiles.

* $u_i : A \to \mathbb{R}$ is player $i$'s utility function.

Multi-Agent Decision Theory
-----

Note the differences between a decision problem and a game:

* No states.  

    Utilities are defined over action profiles.  Source of uncertainty: what will the other agents do?
    
* No probabilities.

    How do we apply tools like `maximize_expected_utility`?

Common Knowledge of Rationality
-------

* Common knowledge that $p$: We all know $p$, we know that we know $p$, we know that we know that we know $p$, ...
    - Inifinite iteration: not needed in many cases; useful for normative evaluation
* Generally, the following are assumed to be common knowledge among players of a game:
    - the precise definition of the game (acts, utilities)
    - _that everyone is rational_ (e.g. an expected utility maximizer)

Mutually Assured Destruction (Prisoner's Dilemma)
------


| &#160; | disarm | arm |
| ----- | ----- | ----- |
| disarm | 0, 0 | -10, 10 |
| arm | 10, -10 | -8, -8 |

In [3]:
import numpy as np
import itertools

# we define an n-player game as an n+1-dimensional array
# dimensions 0 to n-1 correspond to the actions available to each agent
# dimension n gives the utilities for each agent of the corresponding action profile
MAD = np.array([
    # row 1
    [[0, 0], [-10, 10]],
    # row 2
    [[10, -10], [-8, -8]]
])

def action_profiles(game):
    # last dimension is # of players, so we ignore it
    return itertools.product(*[range(num_actions) for num_actions in game.shape[:-1]])

print(list(action_profiles(MAD)))

[(0, 0), (0, 1), (1, 0), (1, 1)]


Solution Concepts, I: Pareto Efficiency
--------

Intuitively: any change in strategies would make _someone_ worse off.

* $s$ Pareto dominates $s'$ iff: $u_i(s) \geq u_i(s')$ for every $i$, and $u_i(s) > u_i(s')$ for some $i$
* $s$ is Pareto optimal iff no strategy profile $s'$ Pareto dominates $s$

In [4]:
def pareto_dominates(game, profile1, profile2):
    return np.all(game[profile1] >= game[profile2]) and np.any(game[profile1] > game[profile2])

def pareto_optimal(game):
    profiles = list(action_profiles(game))
    for profile in profiles:
        if not any(pareto_dominates(game, alt_profile, profile) 
                   for alt_profile in profiles):
            yield profile

print(list(pareto_optimal(MAD)))        

[(0, 0), (0, 1), (1, 0)]


In other words: (disarm, disarm) is Pareto optimal.  So, too, are (disarm, arm) and (arm, disarm).

Solution Concepts, II: Iterated Dominance
-----

* $s_1$ (strictly) dominates $s_2$ for player $i$ iff
    $$u_i(s_1 , s_{-i}) > u_i(s_2, s_{-1})$$
    for all strategy profiles $s$
* $s_1$ weakly dominates $s_2$ for player $i$ iff
    $$u_i(s_1 , s_{-i}) \geq u_i(s_2, s_{-1})$$
    for all strategy profiles $s$ and
    $$u_i(s_1 , s_{-i}) > u_i(s_2, s_{-1})$$
    for some $s$

Solution Concepts, II: Iterated Dominance
-----

Iterated Removal of (strictly/weakly) Dominated Strategies:

* As long as there are dominated strategies in the game $G$:
    - reduce the game by deleting the relevant strategy for the relevant player

Iterated Dominance in MAD
-------

In [5]:
# NOTE: I have only tested this on two player games
def get_utilities(game, player):
    # gets a 1-D array of utilities for `player` for playing `strategy` in `game`
    return game.take(player, axis=-1)

def get_action_utilities(game, player, action):
    return get_utilities(game, player).take(action, axis=player)

In [6]:
def strictly_dominates(game, player, action1, action2):
    return np.all(get_action_utilities(game, player, action1) > 
                  get_action_utilities(game, player, action2))

print(strictly_dominates(MAD, 0, 0, 1))
# print(strictly_dominates(MAD, 0, 1, 0))
# print(strictly_dominates(MAD, 1, 0, 1))
# print(strictly_dominates(MAD, 1, 1, 0))

False


`arm` strictly dominates `disarm` for both players, so the only strategy profile that survives iterated removal is (arm, arm)

The Guessing Game
-----

> You all must guess a number between 1 and 100.  You win (and get utility $1$) if your guess is the closest to 2/3 of the average of everyone's guess.  Everyone else gets utility $0$. 

What is your guess?

* Every number from 68-100 is weakly dominated by some number: 2/3 of the average is always less than or equal to 67.

    So we can delete all choices 68-100 from the game.

* Now, in the reduced game the highest possible average is 67; since 2/3 * 67 = 44 2/3, every number from 46 - 67 will be weakly dominated by some number.

* Now, the highest possible average is ...

* The only strategy to survive iterated removal of _weakly_ dominated strategies is: 1!

Solution Concepts III: Nash Equilibrium
---------

Key idea: no player wants to _unilaterally deviate_.  Given what the other players are doing, no one wants to change what they are doing.

Nash Equilibrium via Best Responses
----------

* $s_i$ is a best response to $s_{-i}$ (i.e. $s_i \in \text{BR}_i(s_{-i})$: 
$$\text{BR}_i(s_{-i}) = \{ s_i \in S_i : u_i(s_i , s_{-i}) \geq u_i(s_i^\prime , s_{-i}) \text{ for every } s_i^\prime \neq s_i \in S_i \}$$

* $s$ is a (weak) _Nash equilibrium_ if and only if for every agent $i$, $s_i \in \text{BR}_i(s_{-i})$

* strict Nash equilibrium: replace $\geq$ with $>$ in the definition of best response

Nash in MAD / PD
---------

* (disarm, disarm) is the only Nash equilibrium in the MAD game.

* This is something of a "paradox": individual rationality seems to inevitably lead to an outcome that is not Pareto optimal and is in fact strictly worse for all parties involved.

Back to the Penalty Kick
----

| &#160; | dive left | dive right |
| ----- | ----- | ----- |
| shoot left | 1, -1 | -1, 1 |
| shoot right | -1, 1 | 1, -1 |

What should Robben do?

Mixed Strategies
-----

* There are _no_ Nash equilibria in _pure_ strategies (i.e. no profile $(a_1, a_2)$ is a n.e., for any $a_1 \in A_1$ and $a_2 \in A_2$

* The set of _mixed_ strategies for player $i$ is $$S_i := \Delta(A_i)$$ where $\Delta(\cdot)$ is the set of probability distributions over a set.

* The set of mixed strategy _profiles_ in a game is $S := S_1 \times \cdots \times S_n$

Utility of a Mixed Strategy Profile
-----

\begin{align*}
u_i(s) &= \mathbb{E}_{p_s}\left[ u_i(a) \right] \\
&= \sum_{a \in A} u_i(a) p_s(a) \\
&= \sum_{a \in A} u_i(a) \prod_{j=1}^n s_j(a_j)
\end{align*}

Finding a Mixed Strategy NE
------

* Key idea: mix among your actions so as to make the other players _indifferent_ among their actions.

* Suppose Robben kicks left with probability $p$ and kicks right with probability $1-p$

\begin{align*}
u_G(\text{dive left}) &= p \cdot -1 + (1-p) \cdot 1 = 1 - 2p \\
u_G(\text{dive right}) &= p \cdot 1 + (1-p) \cdot -1 = 2p - 1 \\
\end{align*}

* Now: $u_G(\text{dive left}) = u_G(\text{dive right})$ if and only if $p = \frac{1}{2}$.

* Similar reasoning shows that Robben is indifferent about whether to kick left or right just in case the goalie mixes between his actions with probability $\frac{1}{2}$

* So: $( (1/2, 1/2), (1/2, 1/2))$ is the only Nash equilibrium of this game.

Nash's Theorem
------

**Theorem (Nash, 1951).** Every finite (in number of players and actions) normal-form game has at least one Nash equilibrium.

Interpretations of Mixed Strategies
------

* Objective chance: when Robben is about to kick, it's as if he flips a coin and chooses which side to go based on that

* Subjective probability: player $i$'s mixed strategy is _everyone else's_ degree of belief about what $i$ will do

* Empirical frequency: how often strategies are played in the limit in _finitely repeated_ versions of the game 

* Population frequency: what percentage of a _population_ plays each pure strategy  
    Much more on this one next week.

Computing Nash Equilibria
------

A Crucial Fact
------

**Theorem.** Let $s_1$ and $s_2$ be mixed strategies for the players in a 2-player game, and $A$ and $B$ their payoff matrices.  Then $s_1 \in \text{BR}_1(s_2)$ if and only if:
$$\text{if } i \in \text{supp}(s_1), \text{ then } (As_2)_i = \max\{ (As_2)_k : k < | A_1 | \}$$

In [7]:
def is_nash(game, row_strategy, col_strategy):
    # _strategy: 1-D numpy array; probability dist over their actions
    row_utilities = get_utilities(game, 0)
    col_utilities = get_utilities(game, 1)
    
    # get expected utilities
    row_expected = np.dot(row_utilities, col_strategy)
    col_expected = np.dot(col_utilities.T, row_strategy)
    
    # support = non-zero probability assigned
    row_support = np.nonzero(row_strategy)
    col_support = np.nonzero(col_strategy)
    
    return (row_expected.max() == row_expected[row_support].max() and
            col_expected.max() == col_expected[col_support].max())

print(is_nash(MAD, [0, 1], [1, 0]))
print(is_nash(MAD, [0, 1], [0, 1]))

False
True


Support Enumeration Methods
--------

1. For every $k$ up to $\min(|A_1|, |A_2|)$:
    1. For every $I \subset A_1, J \subset A_2$ s.t. $|I| = |J| = k$:
        1. Solve the system of equations for $p$ (and same for the other agent):
        \begin{align*}
        &\sum_{a_2 \in J} p(a_2)u_1(a, a_2) = v && &\forall a \in I \\
        &\sum_{a_2 \in J} p(a_2)u_1(a, a_2) \leq v && &\forall a \notin I \\
        &p(a) \geq 0 && &\forall a \in I \\
        &p(a) = 0 && &\forall a \notin I \\
        &\sum_{a \in I} p(a) = 1
        \end{align*}
        
        
See section 4.2.3 of Shohan and Leyton-Brown (linked on homepage of this repository) for more details, and the rest of that chapter for fancier algorithms.

Wrapping up
-----

* Normal form games model _multiagent decision theory_ (so far, in simultaneous games)
* Applies to a wide range of interactive contexts
* Solution concepts: Pareto optimality, iterated removal of dominant strategies, Nash equilibrium
    - Conflict between "group" and "individual" rationality
* Next time: sequential structure, uncertainty about what game is being played